In [1]:
%cd ..

D:\Programming\Research\Thesis\paramsum


In [30]:
import pandas as pd

import string

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=8)

from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt

from IPython.display import display

with open('./data/ne_list.txt', 'rt', encoding='UTF-8') as f:
    named_entities = f.read().split('\n')
    print('\n', len(named_entities), named_entities[:10])

from sklearn.model_selection import train_test_split

SEED = 42
    

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/

 154324 ['Кинг', 'Дарабонт', 'Перси', 'Депрессия', 'Делакруа', 'Майкл Кларк Дункан', 'Фрэнк Дарабонт', 'Стивен Кинг', 'Том Хэнкс', 'Том']


In [3]:
'dsfёэыаtеёэыавм'.replace('ё', 'е')

'dsfеэыаtееэыавм'

In [4]:
import re
from typing import List

punct = list(string.punctuation)
punct.extend(['..', '...', '!!!', '«', '»'])

def str_clean(review: str, char_clean: bool = False, clean_mark_number_patterns: bool = False, lowercase: bool = False, replace_yo: bool = False):
    import re

    if char_clean:
        _html_pattern = r'<[^<]+?>'
        _escape_pattern = r'[\n|\r|\b]'
        review = re.sub(_html_pattern, '  ', review)
        review = re.sub(_escape_pattern, '  ', review)

    if clean_mark_number_patterns:
        mark_number_pattern = r'\d{1,2} из \d{1,2}'
        review = re.sub(mark_number_pattern, '  ', review)

    if replace_yo:
        review = review.replace('ё', 'е')
        review = review.replace('Ё', 'Е')

    if lowercase:
        review = review.lower()


    return review
    

def per_token_clean(
        review: str, lowercase: bool = False, tokenizer_type: str = 'razdel', lemmatizer_type: str = None, 
        stopwords: List[str] = None, punctuation: List[str] = None, named_entities: List[str] = None,
        replace_yo=False, tagger=None, lemmatizer=None 
    ) -> str:

    if tokenizer_type == 'razdel':
        import razdel
        review_tokens = [token.text for token in list(razdel.tokenize(review))]

    elif tokenizer_type == 'TreebankWordTokenizer':
        from nltk.tokenize import TreebankWordTokenizer
        review_tokens = TreebankWordTokenizer().tokenize(review)

    elif tokenizer_type == 'rutokenizer':
        import rutokenizer
        tokenizer = rutokenizer.Tokenizer()
        tokenizer.load()
        review_tokens = tokenizer.tokenize(review)
    else:
        raise NotImplementedError('Unknown tokenizer')

    if punctuation:
        review_tokens = [token for token in review_tokens if token not in punctuation]
        
    if named_entities:
        review_tokens = [token for token in review_tokens if token not in named_entities]

    if lemmatizer_type:
        if lemmatizer_type == 'pymorphy3':
            import pymorphy3
            morph = pymorphy3.MorphAnalyzer(lang='ru')
            review_tokens = [morph.parse(token)[0].normal_form for token in review_tokens]

        elif lemmatizer_type == 'rulemma':
            tags = tagger.tag(review_tokens)
            lemmas = lemmatizer.lemmatize(tags)
            review_tokens = list(map(lambda tpl: tpl[2], lemmas))
        else:
            raise NotImplementedError('Unknown lemmatizer')

    if named_entities:
        review_tokens = [token for token in review_tokens if token not in named_entities]

    if stopwords:
        review_tokens = [token for token in review_tokens if token not in stopwords]

    review = ' '.join(review_tokens)

    if replace_yo:
        review = review.replace('ё', 'е')
        review = review.replace('Ё', 'Е')

    if lowercase:
        review = review.lower()

    return review

In [5]:
reviews = pd.read_csv('./data/reviews.csv', index_col=0)
reviews

,review,kinopoiskId,type,date,positiveRating,negativeRating,author,title,film_id
0,«Зеленую милю» я смотрела два раза: 10 лет наз...,3221833,NEUTRAL,2023-01-28T20:58:59,4,7,Yanchessa,NaN,435
1,Период конца девяностых годов-начало двухтысяч...,3146204,POSITIVE,2022-05-06T09:26:39,13,2,Fozzy,Магия или реальность,435
2,"Очень сложно писать рецензию на этот фильм, та...",3120334,POSITIVE,2022-01-24T21:36:34,21,2,Denis2oo3,"Что происходит на миле, остается на миле",435
3,Любимая многими миллионами ценителями киноиску...,3104371,POSITIVE,2021-12-19T09:09:50,10,2,kingwayne,Живи долго и счастливо!,435
4,В нашем мире существует много разных фильмов. ...,3096653,POSITIVE,2021-12-01T05:31:18,18,2,Юлия Березина - 1765,Шедевр!,435
...,...,...,...,...,...,...,...,...,...
90642,Фильм «Ламборгини: Человек-легенда» снят в 202...,3214599,NEGATIVE,2023-01-09T01:55:11,3,1,Georgy_Olegovich,Марксисткий пасквиль против Ламборгини,969760
90643,"Эй, рагацци, вы это серьёзно, ТАК показывать и...",3211347,NEGATIVE,2022-12-29T19:07:13,48,4,Rosinanto,Американизированная версия итальянской биографии.,969760
90644,"Вообще, говоря о байопиках, стоит отметить, чт...",3209833,NEGATIVE,2022-12-24T19:23:18,72,7,Павел - 5699,Унылая повесть про яркую личность,969760
90645,"Визуальное качество, впрочем, также не безупре...",3201042,NEUTRAL,2022-11-26T19:49:37,309,28,devalmont,Ферруччо Ламборгини,969760


In [6]:
# cur_reviews = reviews.query('film_id == 405609')[['review']]
cur_reviews = reviews[['review']]
cur_reviews

,review
0,«Зеленую милю» я смотрела два раза: 10 лет наз...
1,Период конца девяностых годов-начало двухтысяч...
2,"Очень сложно писать рецензию на этот фильм, та..."
3,Любимая многими миллионами ценителями киноиску...
4,В нашем мире существует много разных фильмов. ...
...,...
90642,Фильм «Ламборгини: Человек-легенда» снят в 202...
90643,"Эй, рагацци, вы это серьёзно, ТАК показывать и..."
90644,"Вообще, говоря о байопиках, стоит отметить, чт..."
90645,"Визуальное качество, впрочем, также не безупре..."


In [7]:
cur_reviews.iloc[2, 0]

'Очень сложно писать рецензию на этот фильм, так как это просто нужно видеть. Это один из самых эмоциональных и сильных фильмов за всю историю кинематографа, который заслужено, на момент написания рецензии,\xa0 находится на первом месте по оценкам.         Определенно картина обязана своим успехом первоисточнику. Работа снята по мотивам известного писателя Стивена Кинга. Но взять первоисточник это лишь фундамент, нужно также грамотно его реализовать и с этим создатели справились превосходно.         Главной заслугой ленты я считаю драматургию. Я считаю, и думаю многие согласятся, что вызвать сильную скорбь и грусть гораздо сложнее чем улыбку. Для этого нужно проделать большую работу над персонажами, чтобы им хотелось сопереживать и выстроить грамотную историю. И с этими двумя аспектами у картины нет никаких проблем. Режиссер ленты сумел правильно перенести историю на экран, используя хорошие ходы и оставляя детали вместе с зацепками. Произведение идёт три часа и за это время мы успевае

In [8]:
cur_reviews['review'] = cur_reviews['review'].parallel_apply(
    str_clean,
    char_clean = True,
    clean_mark_number_patterns = True,
    lowercase = False,  
    replace_yo = True
)

display(cur_reviews)

C:\Users\Yaroslav Pristalov\AppData\Local\Temp\ipykernel_32152\1314494033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_reviews['review'] = cur_reviews['review'].parallel_apply(


,review
0,«Зеленую милю» я смотрела два раза: 10 лет наз...
1,Период конца девяностых годов-начало двухтысяч...
2,"Очень сложно писать рецензию на этот фильм, та..."
3,Любимая многими миллионами ценителями киноиску...
4,В нашем мире существует много разных фильмов. ...
...,...
90642,Фильм «Ламборгини: Человек-легенда» снят в 202...
90643,"Эй, рагацци, вы это серьезно, ТАК показывать и..."
90644,"Вообще, говоря о байопиках, стоит отметить, чт..."
90645,"Визуальное качество, впрочем, также не безупре..."


In [9]:
cur_reviews.iloc[2, 0]

'Очень сложно писать рецензию на этот фильм, так как это просто нужно видеть. Это один из самых эмоциональных и сильных фильмов за всю историю кинематографа, который заслужено, на момент написания рецензии,\xa0 находится на первом месте по оценкам.         Определенно картина обязана своим успехом первоисточнику. Работа снята по мотивам известного писателя Стивена Кинга. Но взять первоисточник это лишь фундамент, нужно также грамотно его реализовать и с этим создатели справились превосходно.         Главной заслугой ленты я считаю драматургию. Я считаю, и думаю многие согласятся, что вызвать сильную скорбь и грусть гораздо сложнее чем улыбку. Для этого нужно проделать большую работу над персонажами, чтобы им хотелось сопереживать и выстроить грамотную историю. И с этими двумя аспектами у картины нет никаких проблем. Режиссер ленты сумел правильно перенести историю на экран, используя хорошие ходы и оставляя детали вместе с зацепками. Произведение идет три часа и за это время мы успевае

In [10]:
from razdel import sentenize

all_sents = []

for review in tqdm(cur_reviews['review']):
    all_sents.extend(list(map(lambda substr_obj: substr_obj.text, list(sentenize(review)))))

all_sents

  0%|          | 0/90647 [00:00<?, ?it/s]

['«Зеленую милю» я смотрела два раза: 10 лет назад и сейчас.',
 'Первый раз у меня было примерно такое впечатление, о котором говорят большинство зрителей.',
 'Я плакала над глубокой и грустной историей, которая надолго оставила след в душе, тем более, что я была подростком.          Несколько лет назад прочитала оригинальный роман Кинга и периодически думала, насколько хороша и точна экранизация (хотя сам фильм я уже успела подзабыть, некоторые моменты еще были живы в памяти).',
 'Сейчас же я решила освежить впечатления и пощекотать эмоции, но это парадоксальным образом не сработало.',
 'После взросления и прочтения книги фильм Дарабонта показался мне плоским, как блин, и даже немного не раскрывающим темы, на которые замахивается.',
 'Это довольно странно, учитывая, что фильм идет три часа, при этом являясь экранизацией не такой уж и большой книги.',
 'Темы расизма, злоупотребления властью, жизни и смерти — все это отдается на откуп зрительскому восприятию.',
 'Особенно нелепа вся эта

In [11]:
cur_reviews = pd.DataFrame(all_sents, columns=['review'])
cur_reviews

,review
0,«Зеленую милю» я смотрела два раза: 10 лет наз...
1,Первый раз у меня было примерно такое впечатле...
2,"Я плакала над глубокой и грустной историей, ко..."
3,Сейчас же я решила освежить впечатления и поще...
4,После взросления и прочтения книги фильм Дараб...
...,...
2248016,"Некоторые сцены по задумке Бертона должны, по ..."
2248017,Толкование официальных иллюстраций к книгам Кэ...
2248018,"Картинка с Бармаглотом, которая в XIX веке воз..."
2248019,"Зная, насколько книга опережала свое время, не..."


In [12]:
cur_reviews.review.str.contains('ё').any(), cur_reviews.review.str.contains('Ё').any()

(False, False)

In [14]:
cur_reviews['review'][:100].parallel_apply(
    per_token_clean,
    lowercase = True,
    tokenizer_type = 'razdel',
    lemmatizer_type = 'pymorphy3',
    stopwords = russian_stopwords,
    punctuation = punct,
    named_entities = named_entities,
    replace_yo = True
)

0                    зеленый миля смотреть 10 год назад
1     примерно впечатление который говорить большинс...
2     плакать глубокий грустный история который надо...
3     решить освежить впечатление пощекотать эмоция ...
4     взросление прочтение книга фильм показаться пл...
                            ...                        
95                       надзиратель уэтмор садист псих
96      бесить весь миля еще сильный раздражать зритель
97    аккуратно причесать отгладить мечтать увидеть ...
98                          миля это 3-й часовой шедевр
99                         человек конец сдержать слеза
Name: review, Length: 100, dtype: object

In [15]:
cur_reviews['review'] = cur_reviews['review'].parallel_apply(
    per_token_clean,
    lowercase = True,
    tokenizer_type = 'razdel',
    lemmatizer_type = 'pymorphy3',
    stopwords = russian_stopwords,
    punctuation = punct,
    named_entities = named_entities,
    replace_yo = True
)

In [16]:
cur_reviews

,review
0,зеленый миля смотреть 10 год назад
1,примерно впечатление который говорить большинс...
2,плакать глубокий грустный история который надо...
3,решить освежить впечатление пощекотать эмоция ...
4,взросление прочтение книга фильм показаться пл...
...,...
2248016,некоторый сцена задумка должный идея вызывать ...
2248017,толкование официальный иллюстрация книга кэрро...
2248018,картинка который xix век возмущать родитель из...
2248019,знать насколько книга опережать свой время нев...


In [17]:
cur_reviews.review[:50].str.cat(sep=' ')

'зеленый миля смотреть 10 год назад примерно впечатление который говорить большинство зритель плакать глубокий грустный история который надолго оставить след душа подросток несколько год назад прочитать оригинальный роман кинг периодически думать насколько хороший точный экранизация хотя фильм успеть подзабыть некоторый момент еще живой память решить освежить впечатление пощекотать эмоция это парадоксальный образ сработать взросление прочтение книга фильм показаться плоский блин немного раскрывать тема который замахиваться довольно странно учитывать фильм идти час это являться экранизация большой книга тема расизм злоупотребление власть жизнь смерть — все это отдаваться откуп зрительский восприятие нелепый весь линия травля весь фильм толком непонятно сослуживец ненавидеть хотеть убить милый потенциально вредный животное работать блат время депрессия ага честно сюжетный линия выглядеть разборка старшеклассник работа тюремный блок кстати ради больший слезовыжималка сценарист умолчать пр

In [18]:
cur_reviews.to_csv('./preprocessed_reviews.csv', index=False)

In [25]:
cur_reviews

,review
0,зеленый миля смотреть 10 год назад
1,примерно впечатление который говорить большинс...
2,плакать глубокий грустный история который надо...
3,решить освежить впечатление пощекотать эмоция ...
4,взросление прочтение книга фильм показаться пл...
...,...
2248016,некоторый сцена задумка должный идея вызывать ...
2248017,толкование официальный иллюстрация книга кэрро...
2248018,картинка который xix век возмущать родитель из...
2248019,знать насколько книга опережать свой время нев...


In [32]:
X_train, X_test = train_test_split(cur_reviews, random_state=SEED, test_size=0.02)

display(X_train)
display(X_test)

,review
259488,реклама видеть еще лето зацепить захватывать с...
1689047,умело передавать зритель порция адреналин держ...
1899527,это чванство сторона покрыть карта простой объ...
568020,считать имя это верх творческий личность
809477,ранее известный режиссер фильм-мясорубка назва...
...,...
732180,фильм по-моему это создать отличный атмосфера ...
110268,вселенная властелин кольцо масштабный безграни...
1692743,минимум эмоция сцена казаться напрашиваться
2229084,никто нигде ждать


,review
1895742,взять основа идея обыграть еще блестяще миниму...
1241220,настоящий человек
952763,идея фильм
132948,явно что-то фильм доставать мочь масштабность
566879,радовать графика
...,...
1265313,определенный вещь который позволять сделать пр...
1517670,честно признаться выбор главный роль обрадовать
40921,девушка поселиться сердце главное герой являть...
219034,задира


In [33]:
with open('../repos/Attention-Based-Aspect-Extraction/preprocessed_data/review/train.txt', 'wt', encoding='UTF-8') as f:
    for line in tqdm(X_train['review']):
        if len(line.split()) > 1:
            f.write(line + '\n')

  0%|          | 0/2203060 [00:00<?, ?it/s]

In [34]:
with open('../repos/Attention-Based-Aspect-Extraction/preprocessed_data/review/test.txt', 'wt', encoding='UTF-8') as f:
    for line in tqdm(X_test['review']):
        if len(line.split()) > 1:
            f.write(line + '\n')

  0%|          | 0/44961 [00:00<?, ?it/s]

In [19]:
with open('../repos/Attention-Based-Aspect-Extraction/preprocessed_data/review/train.txt', 'wt', encoding='UTF-8') as f:
    for line in tqdm(cur_reviews['review']):
        if len(line.split()) > 1:
            f.write(line + '\n')

  0%|          | 0/2248021 [00:00<?, ?it/s]

In [20]:
from collections import Counter

In [21]:
counter = dict(Counter(cur_reviews['review'].str.cat(sep=' ').split()))
counter

{'зеленый': 3392,
 'миля': 1210,
 'смотреть': 61113,
 '10': 6134,
 'год': 61572,
 'назад': 5786,
 'примерно': 2443,
 'впечатление': 18454,
 'который': 259861,
 'говорить': 42382,
 'большинство': 8275,
 'зритель': 53844,
 'плакать': 3985,
 'глубокий': 10236,
 'грустный': 2953,
 'история': 66697,
 'надолго': 1706,
 'оставить': 10066,
 'след': 1847,
 'душа': 22158,
 'подросток': 3547,
 'несколько': 20195,
 'прочитать': 4611,
 'оригинальный': 9514,
 'роман': 8102,
 'кинг': 2201,
 'периодически': 1168,
 'думать': 23593,
 'насколько': 6336,
 'хороший': 100139,
 'точный': 1689,
 'экранизация': 11184,
 'хотя': 25157,
 'фильм': 441189,
 'успеть': 3451,
 'подзабыть': 163,
 'некоторый': 21992,
 'момент': 42596,
 'еще': 76216,
 'живой': 10310,
 'память': 7443,
 'решить': 13297,
 'освежить': 229,
 'пощекотать': 144,
 'эмоция': 21051,
 'это': 368218,
 'парадоксальный': 591,
 'образ': 30321,
 'сработать': 730,
 'взросление': 861,
 'прочтение': 1430,
 'книга': 34468,
 'показаться': 8916,
 'плоский': 1

In [22]:
words = []
counts = []

for k, v in counter.items():
    words.append(k)
    counts.append(v)

In [23]:
word_stats = pd.DataFrame(data={'words': words, 'counts': counts})
display(word_stats)

display(word_stats.query('counts < 3'))

,words,counts
0,зеленый,3392
1,миля,1210
2,смотреть,61113
3,10,6134
4,год,61572
...,...,...
194151,ремеслиничество-артиджани,1
194152,подписотый,1
194153,ordine,1
194154,merito,1


,words,counts
179,годов-начало,1
211,иполнить,1
269,делокруа,1
286,патришиа,1
491,гуманистически,2
...,...,...
194151,ремеслиничество-артиджани,1
194152,подписотый,1
194153,ordine,1
194154,merito,1


In [ ]:
word_stats.query('counts < 3')['words'].str.cat(sep=' ')

In [ ]:
cur_reviews.head(20)

,review
0,зелёный миля смотреть 10 год назад
1,первый примерно впечатление который говорить б...
2,плакать глубокий грустный история который надо...
3,решить освежить впечатление пощекотать эмоция ...
4,взросление прочтение книга фильм дарабонт пока...
5,это довольно странно учитывать фильм идти час ...
6,тема расизм злоупотребление власть жизнь смерт...
7,особенно нелепый весь линия травля перси весь ...
8,хотеть убить милый потенциально вредный животное
9,работать блат время депрессия ага


In [ ]:
cur_reviews.shape

(2248341, 1)